In [1]:
#Kernal PCA jupyter notebook 
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split

train = pd.read_csv("../input/train_dummy.csv", header=0, index_col=0)

train_df = pd.read_csv("competition_starter.csv", header=0, index_col=0)

In [2]:
X = train.values
y = train_df['gp_cost_per_registered_patient'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(6505, 926)
(2788, 926)
(6505,)
(2788,)


In [3]:
## Incremental kernal PCA
# from sklearn.decomposition import PCA
# from sklearn.metrics import mean_squared_error
# pca = PCA()
# pca.fit(X)
# cumsum = np.cumsum(pca.explained_variance_ratio_)
# d = np.argmax(cumsum >= 0.99) + 1

# pca = PCA(n_components = d)
# X_train_red = pca.fit_transform(X_train)
# X_test_red = pca.fit_transform(X_test)
# l1 = LGBM.LGBMRegressor(objective='regression',num_leaves=5,
#                               learning_rate=0.05, n_estimators=720,
#                               max_bin = 55, bagging_fraction = 0.8,
#                               bagging_freq = 5, feature_fraction = 0.2319,
#                               feature_fraction_seed=9, bagging_seed=9,
#                               min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
# l2 = LGBM.LGBMRegressor(objective='regression',num_leaves=5,
#                               learning_rate=0.05, n_estimators=720, 
#                               max_bin = 55, bagging_fraction = 0.8, 
#                               bagging_freq = 5, feature_fraction = 0.2319,
#                               feature_fraction_seed=9, bagging_seed=9,
#                               min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

# l1.fit(X_train, y_train)
# l2.fit(X_train_red, y_train)

# pred1 = l1.predict(X_test)
# pred2 = l2.predict(X_test_red)
# print(X_test_red.shape)
# print(y_test.shape)

# print("model 1 acc: ", mean_squared_error(pred1, y_test))

# print("model 2 acc: ", mean_squared_error(pred2, y_test))

In [4]:
# #Grid Search Light GBM
# import lightgbm as LGBM
# from sklearn.model_selection import GridSearchCV
# from sklearn.pipeline import Pipeline
# from sklearn.decomposition import KernelPCA

# kernels = ['rbf', 'sigmoid']

# kernel_clf = Pipeline([('KPCA', KernelPCA(n_components = 900, n_jobs = -1)),
#                        ('lgbm', LGBM.LGBMRegressor(objective='regression',num_leaves=5,
#                               learning_rate=0.05, n_estimators=720, 
#                               max_bin = 55, bagging_fraction = 0.8, 
#                               bagging_freq = 5, feature_fraction = 0.2319,
#                               feature_fraction_seed=9, bagging_seed=9,
#                               min_data_in_leaf =6, min_sum_hessian_in_leaf = 11))])

# param_grid = [{
#     'KPCA__gamma': np.linspace(0.9, 1, 10),
#     'KPCA__kernel': kernels
# }]

# grid_search = GridSearchCV(kernel_clf, param_grid, cv = 3)
# grid_search.fit(X, y)
# grid_search.best_params_

In [5]:
#Grid Search Light GBM
import lightgbm as LGBM
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

lgm = LGBM.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720, 
                              max_bin = 55, bagging_fraction = 0.8, 
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

param_dist = {'learning_rate': np.linspace(0.01, 0.1, 10),
              'max_bin': sp_randint(40, 60),
              'bagging_freq': sp_randint(1, 10),
              'bagging_fraction': np.linspace(0.2, 0.1, 10),
              'feature_fraction': np.linspace(0.2,0.1,10),
              'min_data_in_leaf': sp_randint(3,10),
              'num_leaves': sp_randint(4, 10)}

param_grid = [{
    'learning_rate': np.linspace(0.01, 0.1, 10),
    'max_bins': np.linspace(50, 60, 10),
    'baggin_frequency': np.linspace(1,10,10),
    'feature_fraction': np.linspace(0.2,0.3, 10),
}]

n_iter = 50
rand_search = RandomizedSearchCV(lgm, param_distributions=param_dist, n_iter=n_iter, cv = 3)
rand_search.fit(X, y)
print(rand_search.cv_results_)
print("Best Parameters - ", rand_search.best_params_)

{'std_fit_time': array([ 0.00666995,  0.00326355,  0.0029184 ,  0.00123614,  0.00073872,
        0.00214064,  0.00260045,  0.00142305,  0.00121632,  0.00045337,
        0.0031411 ,  0.00924194,  0.00857614,  0.00461767,  0.00382641,
        0.00393504,  0.00170211,  0.00266178,  0.00722692,  0.00128835,
        0.0011118 ,  0.00140271,  0.06257075,  0.00174135,  0.00228474,
        0.00871913,  0.00351348,  0.00107573,  0.00059903,  0.00245479,
        0.00243862,  0.00097616,  0.00181111,  0.00214596,  0.00560576,
        0.00244426,  0.00295484,  0.00206705,  0.00173811,  0.0031491 ,
        0.0034064 ,  0.00285433,  0.00369602,  0.00319803,  0.00282252,
        0.0026327 ,  0.00273376,  0.00122508,  0.00287983,  0.00415891]), 'split1_train_score': array([ 0.8316229 ,  0.80475014,  0.83507663,  0.79277781,  0.78487736,
        0.81939144,  0.81393403,  0.78017643,  0.77906845,  0.78517788,
        0.7939719 ,  0.83695003,  0.83283945,  0.80785669,  0.84993694,
        0.79874603,  0.

In [6]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import xgboost as xgb
import lightgbm as lgb

/home/mark/anaconda3/envs/python35/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1, nthread = -1)

param_dist = {'colsample_bytree': np.linspace(0.3, 0.5, 20),
              'gamma': np.linspace(0.03, 0.05, 20),
              'learning_rate': np.linspace(0.01, 0.1, 10),
              'max_depth': sp_randint(3, 10),
              'reg_alpha': np.linspace(0.1, 1, 20),
              'reg_lambda': np.linspace(0.1, 1, 20),
              'subsample': np.linspace(0.1, 1, 20)}

n_iter = 10
rand_search = RandomizedSearchCV(model_xgb, param_distributions=param_dist, n_iter=n_iter, cv=3)
rand_search.fit(X, y)
print(rand_search.cv_results_)
print("Best Parameters - ", rand_search.best_params_)

{'std_fit_time': array([ 0.00934529,  0.01225036,  0.00128027,  0.00937503,  0.01547522,
        0.0017511 ,  0.00113125,  0.02675971,  0.1530404 ,  0.02871579]), 'param_colsample_bytree': masked_array(data = [0.32105263157894737 0.33157894736842103 0.32105263157894737
 0.48947368421052628 0.40526315789473683 0.47894736842105262
 0.47894736842105262 0.43684210526315792 0.44736842105263153
 0.46842105263157896],
             mask = [False False False False False False False False False False],
       fill_value = ?)
, 'split1_train_score': array([ 0.68102677,  0.82383134,  0.69665507,  0.84159301,  0.76082452,
       -2.8985096 ,  0.75046991, -2.84135986, -2.97121587,  0.88898821]), 'split0_train_score': array([ 0.60020101,  0.74490544,  0.62783083,  0.78250155,  0.65503455,
       -4.67669896,  0.68133422, -4.66640546, -4.74889575,  0.8448835 ]), 'params': ({'subsample': 0.81052631578947365, 'max_depth': 3, 'reg_alpha': 0.76315789473684215, 'reg_lambda': 0.62105263157894741, 'gamma': 0

In [16]:
GBoost = GradientBoostingRegressor(learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

param_dist = {'min_samples_leaf': sp_randint(3, 20),
              'min_samples_split': sp_randint(3, 20),
              'learning_rate': np.linspace(0.01, 0.1, 10),
              'max_depth': sp_randint(3, 10)}

n_iter = 10
rand_search = RandomizedSearchCV(GBoost, param_distributions=param_dist, n_iter=n_iter, cv=3)
rand_search.fit(X, y)
print(rand_search.cv_results_)
print("Best Parameters - ", rand_search.best_params_)

{'mean_score_time': array([ 0.02211873,  0.01861477,  0.01144139,  0.00850081,  0.01561824,
        0.01642553,  0.01925659,  0.00978692,  0.0155743 ,  0.01023348]), 'std_fit_time': array([ 0.25057354,  0.05545954,  0.00778544,  0.00451076,  0.11896419,
        0.06341764,  0.19138096,  0.00569514,  0.12039097,  0.00527848]), 'param_min_samples_split': masked_array(data = [17 14 17 5 7 3 12 18 10 7],
             mask = [False False False False False False False False False False],
       fill_value = ?)
, 'param_learning_rate': masked_array(data = [0.030000000000000006 0.080000000000000002 0.060000000000000012 0.01
 0.020000000000000004 0.070000000000000007 0.040000000000000008
 0.070000000000000007 0.020000000000000004 0.10000000000000001],
             mask = [False False False False False False False False False False],
       fill_value = ?)
, 'mean_train_score': array([ 0.68174669,  0.81369681,  0.67794274,  0.31574908,  0.66445481,
        0.78582476,  0.78216767,  0.64193266,  